# Fragrantica perfume review clasifier (LSTM with lemmatizated)

In [11]:
import pandas as pd
import requests
from os import path
from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, LSTM, Embedding
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [12]:
# Model Name
MODEL_NAME = 'lstm_with_lemmatizated_12'

# HyperParameters
PAD_LEN = 200
NUM_WORDS = 1000
EMBEDDING = 100
BATCH_SIZE = 512


In [13]:
data_path = '../dataset/dataset_210626_215600.csv'
data_exist = path.exists(data_path)

if not data_exist:
    url = 'https://kyuuuw-nlp-dataset.s3.ap-northeast-2.amazonaws.com/fragrantica/dataset_210626_215600.csv'
    r = requests.get(url, allow_redirects=True)
    open(data_path, 'w').write(r.content)

In [14]:
data = pd.read_csv(data_path)

X_data = data['lemmatizated']
y_data = data['label']

print(len(X_data))
print(len(y_data))


##### 토큰화 및 인덱스 부여

74779
74779


In [15]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_data)
sequences = tokenizer.texts_to_sequences(X_data)

print(X_data[0])
print(len(sequences[0]))

['get', 'sample', 'today', 'year', 'old', 'daughter', 'think', 'smell', 'spray', 'card', 'rotten', 'fish', 'nearly', 'throw', 'immediately', 'however', 'later', 'even', 'decide', 'give', 'fair', 'trial', 'spray', 'crook', 'elbow', 'rot', 'fish', 'smell', 'time', 'get', 'definite', 'bit', 'funk', 'almost', 'urine', 'scent', 'minute', 'blossom', 'gorgeous', 'smooth', 'woody', 'ambery', 'clean', 'warm', 'jasmine', 'daughter', 'didnt', 'even', 'believe', 'tell', 'fragrance', 'smell', 'earlier', 'lol', 'immediately', 'comment', 'fresh', 'clean', 'vibe', 'agree', 'soapy', 'hint', 'powdery', 'way', 'obsess', 'cant', 'stop', 'smelling', 'arm', 'glad', 'give', 'try', 'wait', 'buy', 'full', 'bottle']
76


##### 인덱스별 단어 표시

In [16]:
word_to_index = tokenizer.word_index
print(word_to_index)

{"'smell'": 1, "'like'": 2, "'scent'": 3, "'fragrance'": 4, "n't": 5, "'get'": 6, "'love'": 7, "'wear'": 8, "'note'": 9, "'perfume'": 10, "'bottle'": 11, "'sweet'": 12, "'really'": 13, "'think'": 14, "'time'": 15, "'would'": 16, "'say'": 17, "'well'": 18, "'skin'": 19, "'make'": 20, "'first'": 21, "'buy'": 22, "'hour'": 23, "'much'": 24, "'spray'": 25, "'day'": 26, "'last'": 27, "'try'": 28, "'vanilla'": 29, "'still'": 30, "'even'": 31, "'people'": 32, "'longevity'": 33, "'feel'": 34, "'something'": 35, "'want'": 36, "'strong'": 37, "'find'": 38, "'give'": 39, "'great'": 40, "'could'": 41, "'also'": 42, "'know'": 43, "'way'": 44, "'use'": 45, "'fresh'": 46, "'come'": 47, "'dry'": 48, "'nice'": 49, "'bit'": 50, "'long'": 51, "'many'": 52, "'little'": 53, "'year'": 54, "'ve": 55, "'definitely'": 56, "'open'": 57, "'sillage'": 58, "'review'": 59, "'work'": 60, "'beautiful'": 61, "'see'": 62, "'best'": 63, "'woman'": 64, "'quite'": 65, "'lot'": 66, "'warm'": 67, "'never'": 68, "'compliment

##### 빈도수 분석

In [17]:
threshold = 2
total_cnt = len(word_to_index) # 총 단어의 수
rare_cnt = 0 # 등장 빈도 수가 threshold 보다 작은 단어의 개수를 카운트
total_freq = 0 # 훈련 데이터의 전체 단어 빈도수 총 합
rare_freq = 0

for key, value in tokenizer.word_counts.items():
    total_freq = total_freq + value

    if value < threshold:
        rare_cnt = rare_cnt + 1
        rare_freq = rare_freq + value

print(f"등장 빈도가 {threshold}번 이하인 희귀 단어의 수: {rare_cnt}")
print(f"단어 집합(vocabulary)에서 희귀 단어의 비율: {rare_cnt / total_cnt}" )
print(f"전체 등장 빈도에서 희귀 단어 등장 빈도 비율: {rare_freq / total_freq}")

등장 빈도가 2번 이하인 희귀 단어의 수: 43039
단어 집합(vocabulary)에서 희귀 단어의 비율: 0.5753877005347594
전체 등장 빈도에서 희귀 단어 등장 빈도 비율: 0.012692410682162147


##### 등장 횟수 상위 50000개의 word 만 사용

In [18]:
tokenizer = Tokenizer(num_words=NUM_WORDS)
tokenizer.fit_on_texts(X_data)
sequences = tokenizer.texts_to_sequences(X_data)

print(sequences[:5])

KeyboardInterrupt: 

##### Split data

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(sequences, y_data)
print(len(X_train))
print(len(y_train))
print(len(X_test))
print(len(y_test))

In [ ]:
print(X_train[0])
print(y_train[0])

In [ ]:
# padding and trimming
X_train = pad_sequences(X_train, maxlen=PAD_LEN)
X_test = pad_sequences(X_test, maxlen=PAD_LEN)

In [ ]:
# One hot encoding
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train[0])

In [ ]:
model = Sequential()
model.add(Embedding(NUM_WORDS, EMBEDDING))
model.add(LSTM(EMBEDDING))
model.add(Dense(4, activation='softmax'))

model.summary()

In [ ]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=4)
mc = ModelCheckpoint(f'../model/{MODEL_NAME}.h5', monitor='val_acc', mode='max')

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

In [ ]:
history = model.fit(X_train, y_train,
                    batch_size=BATCH_SIZE, epochs=30, callbacks=[es, mc],
                    validation_data=(X_test, y_test))